## Demo

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pandas as pd

bikes = pd.read_csv('./datasets/austin_bikes_final.csv')
bikes

,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,8,2015-03-19 19:12:00,41,Waller & 6th St.,Trinity & 6th Street,30.26461,-97.73049,30.26735,-97.73933
1,468,2016-03-21 21:17:48,7,Waller & 6th St.,Trinity & 6th Street,30.26461,-97.73049,30.26735,-97.73933
2,107,2016-05-29 13:09:31,35,Waller & 6th St.,Trinity & 6th Street,30.26461,-97.73049,30.26735,-97.73933
3,428,2016-02-20 14:40:49,12,Waller & 6th St.,Trinity & 6th Street,30.26461,-97.73049,30.26735,-97.73933
4,572,2015-07-09 16:12:00,4,Waller & 6th St.,Trinity & 6th Street,30.26461,-97.73049,30.26735,-97.73933
...,...,...,...,...,...,...,...,...,...
614423,183,2017-07-11 18:56:51,43,Congress & Cesar Chavez,Congress & Cesar Chavez,30.26332,-97.74508,30.26332,-97.74508
614424,849,2017-07-13 06:02:13,5,Congress & Cesar Chavez,Boardwalk West,30.26332,-97.74508,30.25457,-97.74258
614425,209,2017-07-31 11:20:23,31,Congress & Cesar Chavez,Boardwalk West,30.26332,-97.74508,30.25457,-97.74258
614426,2048,2017-07-21 06:08:30,5,Congress & Cesar Chavez,Boardwalk West,30.26332,-97.74508,30.25457,-97.74258


In [11]:
from jaccard_workload_approximator import JaccardWorkloadApproximator

workload_approximator = JaccardWorkloadApproximator()
workload = workload_approximator.get_workload_approximation(30)

============ starting iteration 1   ============
Created query:
SELECT _id FROM public.austin_bikes_1000 WHERE start_latitude BETWEEN 30.280838931893975 AND 30.285228319610322 AND bikeid BETWEEN 1129.110366328931 AND 1288.8863541010926 AND end_longitude BETWEEN -97.75472365881295 AND -97.74052115610151
iteration took: 1.14 ms
============ starting iteration 2   ============
Created query:
SELECT _id FROM public.austin_bikes_1000 WHERE start_station_name IN ('Brazos & 6th' , '6th & Navasota St.' , 'Republic Square @ Guadalupe & 4th St.' , 'Palmer Auditorium' , 'Pfluger Bridge @ W 2nd Street' , 'Barton Springs Pool' , 'East 6th & Pedernales St.' , 'East 2nd & Pedernales' , 'UT West Mall @ Guadalupe' , 'East 6th at Robert Martinez' , 'Capitol Station / Congress & 11th' , 'Long Center @ South 1st & Riverside' , 'South Congress & Academy' , 'Nueces @ 3rd' , 'East 11th St. at Victory Grill' , '3rd & West' , 'San Jacinto & 8th Street' , 'Guadalupe & 21st' , '17th & Guadalupe' , 'Republic Squa

In [14]:
from saqp_manager import SaqpManager

saqp_manager = SaqpManager([query[0] for query in workload], [query[1] for query in workload], 30)
sample = saqp_manager.get_sample()
sample

Start running LazyGreedy with type UC
============ starting iteration 1   ============
iteration took: 105.14 ms
============ starting iteration 2   ============
iteration took: 246.75 ms
============ starting iteration 3   ============
iteration took: 304.98 ms
============ starting iteration 4   ============
iteration took: 335.44 ms
============ starting iteration 5   ============
iteration took: 331.48 ms
============ starting iteration 6   ============
iteration took: 388.87 ms
============ starting iteration 7   ============
iteration took: 402.62 ms
============ starting iteration 8   ============
iteration took: 386.69 ms
============ starting iteration 9   ============
iteration took: 433.45 ms
============ starting iteration 10   ============
iteration took: 464.90 ms
============ starting iteration 11   ============
iteration took: 513.83 ms
============ starting iteration 12   ============
iteration took: 510.36 ms
============ starting iteration 13   ============
iteration

[{'_id': 614429,
  'bikeid': 693,
  'start_time': datetime.datetime(2016, 3, 20, 19, 46, 36, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=120, name=None)),
  'duration_minutes': 8,
  'start_station_name': '5th & Bowie',
  'end_station_name': '8th & Congress',
  'start_latitude': 30.2696,
  'start_longitude': -97.75332,
  'end_latitude': 30.2698,
  'end_longitude': -97.74186},
 {'_id': 614430,
  'bikeid': 577,
  'start_time': datetime.datetime(2016, 3, 1, 16, 7, 50, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=120, name=None)),
  'duration_minutes': 11,
  'start_station_name': 'Barton Springs @ Kinney Ave',
  'end_station_name': '5th & Bowie',
  'start_latitude': 30.262,
  'start_longitude': -97.76118,
  'end_latitude': 30.2696,
  'end_longitude': -97.75332},
 {'_id': 614431,
  'bikeid': 465,
  'start_time': datetime.datetime(2016, 5, 5, 7, 29, 25, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=180, name=None)),
  'duration_minutes': 13,
  'start_station_name': 'South Congress & Academ

In [15]:
df = pd.DataFrame(sample)

,_id,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,614429,693,2016-03-20 19:46:36+02:00,8,5th & Bowie,8th & Congress,30.26960,-97.75332,30.26980,-97.74186
1,614430,577,2016-03-01 16:07:50+02:00,11,Barton Springs @ Kinney Ave,5th & Bowie,30.26200,-97.76118,30.26960,-97.75332
2,614431,465,2016-05-05 07:29:25+03:00,13,South Congress & Academy,Capitol Station / Congress & 11th,30.25226,-97.74854,30.27260,-97.74127
3,614432,391,2014-06-28 09:12:00+03:00,10,State Capitol Visitors Garage @ San Jacinto & ...,2nd & Congress,30.27336,-97.73805,30.26408,-97.74355
4,614433,970,2017-05-16 17:41:07+03:00,10,6th & Congress,Davis at Rainey Street,30.26822,-97.74285,30.26019,-97.73845
5,614434,961,2014-03-28 18:12:00+03:00,8,Capitol Station / Congress & 11th,City Hall / Lavaca & 2nd,30.27260,-97.74127,30.26476,-97.74678
6,614435,775,2015-03-14 18:12:00+02:00,1,Barton Springs & Riverside,Barton Springs & Riverside,30.25870,-97.74872,30.25870,-97.74872
7,614436,668,2015-03-29 09:12:00+03:00,28,Palmer Auditorium,5th & Bowie,30.25971,-97.75346,30.26960,-97.75332
8,614437,8,2014-10-27 16:12:00+02:00,3,San Jacinto & 8th Street,Convention Center / 4th St. @ MetroRail,30.26912,-97.73986,30.26483,-97.73900
9,614438,37,2015-10-04 17:12:16+03:00,6,Barton Springs @ Kinney Ave,Zilker Park,30.26200,-97.76118,30.26590,-97.76822


In [18]:
from data_access import DataAccess

dataAccess = DataAccess()
dataAccess.select_to_df("SELECT * FROM public.austin_bikes_1000 WHERE end_latitude BETWEEN 30.257 AND 30.264 AND end_longitude BETWEEN -97.762 AND -97.739")

,_id,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,614435,775,2015-03-14 18:12:00+02:00,1,Barton Springs & Riverside,Barton Springs & Riverside,30.25870,-97.74872,30.25870,-97.74872
1,614448,37,2016-03-28 20:36:20+03:00,14,UT West Mall @ Guadalupe,Long Center @ South 1st & Riverside,30.28576,-97.74181,30.25941,-97.74971
2,614467,124,2016-05-29 11:03:32+03:00,53,Rainey St @ Cummings,Barton Springs & Riverside,30.25579,-97.73982,30.25870,-97.74872
3,614470,625,2015-09-07 22:12:09+03:00,19,Barton Springs @ Kinney Ave,Long Center @ South 1st & Riverside,30.26200,-97.76118,30.25941,-97.74971
4,614503,745,2016-10-24 16:15:51+03:00,65,Long Center @ South 1st & Riverside,Long Center @ South 1st & Riverside,30.25941,-97.74971,30.25941,-97.74971
...,...,...,...,...,...,...,...,...,...,...
110,615405,789,2015-11-14 07:12:23+02:00,4,3rd & West,Toomey Rd @ South Lamar,30.26780,-97.75189,30.26304,-97.75824
111,615408,865,2015-10-03 12:12:50+03:00,36,Long Center @ South 1st & Riverside,Toomey Rd @ South Lamar,30.25941,-97.74971,30.26304,-97.75824
112,615409,406,2015-07-04 17:12:00+03:00,10,Rainey St @ Cummings,Barton Springs & Riverside,30.25579,-97.73982,30.25870,-97.74872
113,615416,327,2016-01-31 18:10:23+02:00,12,Convention Center / 4th St. @ MetroRail,Rainey @ River St,30.26483,-97.73900,30.25802,-97.73910
